In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv("../datasets/ml-100k/u.data.csv", sep="\t",header= None)
df.head()

,0,1,2,3
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [3]:
df.columns=["UserID","ItemID","Rating","TimeStamp"]

* UserID ----> identificador de usuario
* ItemID ----> identificador de la película 
* Rating ----> es la valoración 
* TimeStamp--> fecha de la valoración en segundos 

## Representación en forma matricial

In [4]:
n_users = df.UserID.unique().shape[0]
n_users

943

In [5]:
n_items =df.ItemID.unique().shape[0] #número de películas
n_items

1682

Se crea una matriz vacía

In [6]:
ratings = np.zeros((n_users,n_items)) # filas ---> usuarios
                                    #columnas --> películas

In [7]:
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
for row in df.itertuples():# Itere sobre filas de DataFrame como nombres de tuplas. 
    
    ratings[row[1]-1, row[2]-1] = row[3] # Usuarios del data set empiezan en 1 e índices de python empiezan en 0

In [9]:
ratings # Esta ya es la "sparse matrix"

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [10]:
sparsity = float(len(ratings.nonzero()[0]))

sparsity/=(ratings.shape[0]*ratings.shape[1])

sparsity *=100

print("Coeficiente de sparseiedad:{:4.2f}%".format(sparsity))

Coeficiente de sparseiedad:6.30%


### Crear conjuntos de entrenamiento y validación

In [12]:
ratings_train, ratings_test = train_test_split(ratings, test_size=0.3, random_state=42) #ratings--> parámetro

In [13]:
ratings_train.shape # 660 personas / conjunto de train de 70% / semilla aleatoria = 42

(660, 1682)

In [14]:
ratings_test.shape # conjunto de test de 30% / a estos usuarios les vamos a ofrecer una recomendación 

(283, 1682)

#### Recordar que el número de usuarios es de 943 y el número de películas es de 1682

#### Conozco a un usuario que sé qué películas ha visto y qué valoración les ha dado

#### puedo intentar inferir los gustos de un usuario parecido a este a la Netflix

## Filtro colaborativo basado en Usuarios

* Matriz de similaridad entre los usuarios(distancia del coseno)

* Predecir la valoración desconocida de un item *i* para un usuario activo *u* basándonos en la suma ponderada de todas las valoraciones del resto de usuarios para dicho item

* Recomendaremos los nuevos items a los usuarios según lo establecido en los pasos anteriores 

In [17]:
# Calculamos la matriz de similaridad :
sim_matrix = 1- sklearn.metrics.pairwise.cosine_distances(ratings_train)

sim_matrix.shape # la diagonal tiene "unos" porque un usuario es 100 por cien similar a sí mismo

# usuarios parecidos en cuanto  a los items (películas) que vieron 

(660, 660)

In [18]:
sim_matrix # se usan los usuarios del conjunto de entrenamiento para construir esta matriz 
           # la matriz es simétrica
           # la matriz muestra qué tan perecidos con en gustos los ususarios uno respecto a los demás 

array([[1.        , 0.17448349, 0.18009754, ..., 0.13331459, 0.17695593,
        0.21882481],
       [0.17448349, 1.        , 0.07993097, ..., 0.07175808, 0.09552622,
        0.05512655],
       [0.18009754, 0.07993097, 1.        , ..., 0.0191736 , 0.02233385,
        0.10310785],
       ...,
       [0.13331459, 0.07175808, 0.0191736 , ..., 1.        , 0.04853428,
        0.05142508],
       [0.17695593, 0.09552622, 0.02233385, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.21882481, 0.05512655, 0.10310785, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [19]:
users_predictions = sim_matrix.dot(ratings_train) / np.array([np.abs(sim_matrix.sum(axis=1))]).T

#  sim_matrix----> shape(660,660) ---> muestra qué tan parecidos con en gustos los usuarios entre sí dada la distancia coseno
# ratings_train ---> shape--->(660, 1682) ---> es la matriz que contiene usuarios y sus valoraciones


In [20]:
sim_matrix.dot(ratings_train).shape

(660, 1682)

In [21]:
#Cada fila de sim_matrix.dot(ratings_train) se divide entre los valores de: np.array([np.abs(sim_matrix.sum(axis=1))]).T
# i.e ponderamos por todos los usuarios de la misma fila

In [22]:
np.array([np.abs(sim_matrix.sum(axis=1))]).T.shape

(660, 1)

In [23]:
users_predictions # obtenemos la valoración que aproximadamente cada usuario le daría a la película en cuestion
                  #predicción para el conjunto de entrenamiento

array([[2.10259747e+00, 5.86975978e-01, 3.40264192e-01, ...,
        0.00000000e+00, 7.33611460e-03, 6.04379414e-03],
       [1.40999723e+00, 2.91863934e-01, 2.68085289e-01, ...,
        0.00000000e+00, 3.50378592e-03, 2.32963985e-03],
       [1.69014833e+00, 3.13648440e-01, 3.26127887e-01, ...,
        0.00000000e+00, 3.25391767e-03, 1.77210119e-03],
       ...,
       [1.73393747e+00, 4.06719333e-01, 3.21166908e-01, ...,
        0.00000000e+00, 2.71269625e-03, 9.00511411e-03],
       [2.34361031e+00, 8.10544770e-01, 4.73941025e-01, ...,
        0.00000000e+00, 1.01130066e-02, 9.66427605e-03],
       [2.36796969e+00, 5.98146138e-01, 3.85569804e-01, ...,
        0.00000000e+00, 6.39996638e-03, 5.37442746e-03]])

In [25]:
def get_mse(preds,actuals):
    preds = preds[actuals.nonzero()].flatten() # flatten() pasarlo a vector
    actuals = actuals[actuals.nonzero()].flatten()
    return mean_squared_error(preds, actuals)

In [26]:
get_mse(users_predictions, ratings_train)

7.878218313143215

In [27]:
get_mse(users_predictions, ratings_test)

8.745164067978985

### Filtro colaborativo basado en los KNN

In [34]:
k = 10 # Elegido arbiotrariamente # elegiríamos el top 10 se usuarios más parecidos al mío 
# Calculamos los vecinos más cercanos : 

neighbors = NearestNeighbors(n_neighbors=k, metric="cosine") # usamos la similaridad del coseno  

In [35]:
neighbors.fit(ratings_train) # ajustar la matriz de valoraciones con k = 10 vecinos más cercanos 

NearestNeighbors(metric='cosine', n_neighbors=10)

In [37]:
# Hacemos la predicción de los usuarios de ratings_test basándonos en los KNN del conjunto ratings_train:
top_k_distances_test, top_k_users_test = neighbors.kneighbors(ratings_test, return_distance=True)

In [40]:
top_k_distances_test.shape

(283, 10)

In [41]:
top_k_users_test.shape

(283, 10)

In [42]:
# definimos las similitudes asociadas
top_k_sim_test = 1 - top_k_distances_test

In [45]:
top_k_sim_test

array([[0.49992721, 0.44521803, 0.44201167, ..., 0.42703879, 0.42450563,
        0.42356662],
       [0.5016859 , 0.47665064, 0.44275638, ..., 0.40982666, 0.40669109,
        0.40499101],
       [0.603674  , 0.49298626, 0.47762363, ..., 0.43263079, 0.42194133,
        0.42111744],
       ...,
       [0.48251656, 0.47648875, 0.47045863, ..., 0.45992761, 0.45807802,
        0.45344455],
       [0.31585323, 0.28114149, 0.26759732, ..., 0.25128468, 0.24485608,
        0.24437321],
       [0.57844661, 0.55707747, 0.54650895, ..., 0.53240111, 0.52675814,
        0.51468267]])

In [46]:
top_k_distances_test

array([[0.50007279, 0.55478197, 0.55798833, ..., 0.57296121, 0.57549437,
        0.57643338],
       [0.4983141 , 0.52334936, 0.55724362, ..., 0.59017334, 0.59330891,
        0.59500899],
       [0.396326  , 0.50701374, 0.52237637, ..., 0.56736921, 0.57805867,
        0.57888256],
       ...,
       [0.51748344, 0.52351125, 0.52954137, ..., 0.54007239, 0.54192198,
        0.54655545],
       [0.68414677, 0.71885851, 0.73240268, ..., 0.74871532, 0.75514392,
        0.75562679],
       [0.42155339, 0.44292253, 0.45349105, ..., 0.46759889, 0.47324186,
        0.48531733]])

In [47]:
user_predicts_test_k = np.zeros(ratings_test.shape) # predicciones de los k usuarios 
user_predicts_test_k.shape #matriz de ceros

(283, 1682)

In [50]:
top_k_sim_test.shape


(283, 10)

In [52]:
ratings_train[top_k_users_test].shape

(283, 10, 1682)

In [59]:
# Obtenemos las predicciones del conjunto de test

for i in range(ratings_test.shape[0]):# para cada usuario del conjunto de testing
    
    user_predicts_test_k[i,:] = top_k_sim_test[i].T.dot(ratings_train[top_k_users_test[i]])/np.array([np.abs(top_k_sim_test[i].T).sum(axis=0)]).T
     # para normalizar ---> np.sum(top_k_sim_test[i])


In [60]:
user_predicts_test_k

array([[4.00761846, 1.01878588, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.90998322, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [3.7897992 , 1.98636305, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.21755095, 0.29580008, 0.20248756, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [61]:
get_mse(user_predicts_test_k,ratings_test) # Es razonablemente más bajo que el calculado en los videos

4.102396284374268